In [1]:
!pip install jdc

In [2]:
import pandas as pd
import numpy as np
import jdc

In [3]:
class MDP():
  def __init__(self, num_state, num_actions, filename, gamma):
    self.num_states = num_state
    self.num_actions = num_actions
    self.filename = filename
    self.gamma = gamma

    self.stateDic = {}
    self.rewardArr = [[] for i in range(self.num_states)]
    

    self.read_from_file()

In [4]:
%%add_to MDP
def read_from_file(self):
  file = open(self.filename, "r")

  data_obj = []
  
  for line in file:
    data_obj.append(line)

  file.close()

  print(data_obj)

  for i, obj in enumerate(data_obj):
    obj = obj.rstrip()
    obj = obj.split("(")
    obj = "".join(obj)
    obj = obj.split(")")
    obj = "".join(obj)
    obj = obj.split(" ")
    data_obj[i] = obj

  for x, obj in enumerate(data_obj):
    action = {}
    for a in range(self.num_actions):
      i = a+1
      action['a'+str(i)] = []
    self.stateDic[obj[0]] = {"reward":int(obj[1]), "actions": action}
    self.rewardArr[x].append(int(obj[1]))
    

  # self.stateDic["s1"]["actions"]["a1"].append({"s1": 0.509})
  # self.stateDic["s1"]["actions"]["a1"].append({"s2": 0.491})

  # print(data_obj[2])

  for obj in data_obj:
    idx = 3
    for action in obj[2::3]:
      self.stateDic[obj[0]]["actions"][action].append({obj[idx]: float(obj[idx+1])})
      idx += 3
  
  print(self.stateDic)
  print(self.rewardArr)

In [5]:
%%add_to MDP
def Bellman_equation(self):
  """
  E.g., [a1] ==> J^2(s1) = (r(s1) = 5) + ((gamma = 0.8)*( ((Prob(a1, s1) = 0.509) * J^1(s1) = 5)  +  ((Prob(a1, s2) = 0.491) * J^1(s2) = 10))
        [a2] ==> J^2(s1) = (r(s1) = 5) + ((gamma = 0.8)*( ((Prob(a1, s1) = 0.31) * J^1(s1) = 5)  +  ((Prob(a1, s3) = 0.69) * J^1(s3) = -5))
        J^2(s1) = max(a1, a2)

  """

  # states = {"s1": 0, "s2": 1, "s3": 2} # 0: "s1", 1: "s2", 2: "s3"}
  states = {}
  for i in range(self.num_states):
    states["s"+str(i+1)] = i
  print(states) 

  for k in range(1,20): # 20):
    # print("k = ", k)
    for state, st_idx in states.items():
      reward = self.stateDic[state]["reward"]
      # print(state)
      # print(reward)
      tmp_j_si = []
      rew_idx = 0
      for action in self.stateDic[state]["actions"]:
        # print(action)
        num_state_trans = len(self.stateDic[state]["actions"][action])
        j_sum = 0
        for trans_idx in range(num_state_trans):
          s_lst = list(self.stateDic[state]["actions"][action][trans_idx])[0] # get the states inside the action list
          # print("{} -> r = {}".format( s_lst, self.rewardArr[states[s_lst]][k-1] ) )

          j_sum += ( self.stateDic[state]["actions"][action][trans_idx][s_lst] * self.rewardArr[states[s_lst]][k-1] )
        j_si = reward + (self.gamma * j_sum)
        j_si = round(j_si, 3)
        # print(j_si)
        tmp_j_si.append(j_si)
      self.rewardArr[st_idx].append(max(tmp_j_si))


In [6]:
n_state = 3
n_actions = 2
filename = 'test1.in'

# n_state = 10 # 3
# n_actions = 3 # 2
# filename = 'test2.in'
discount = 0.8
mdp = MDP(n_state, n_actions, filename, discount)
mdp.Bellman_equation()

st = {}

for i in range(mdp.num_states):
  st[i] = "s"+str(i+1)
print(st)

for i, val in st.items():
  print("{} ==> {}". format(val, mdp.rewardArr[i]))

['s1 5 (a1 s1 0.509) (a1 s2 0.491) (a2 s1 0.31) (a2 s3 0.69)\n', 's2 10 (a1 s1 0.4) (a1 s2 0.3) (a1 s3 0.3) (a2 s2 0.5) (a2 s3 0.5)\n', 's3 -5 (a1 s1 0.3) (a1 s2 0.3) (a1 s3 0.4) (a2 s1 0.2) (a2 s2 0.8)\n']
{'s1': {'reward': 5, 'actions': {'a1': [{'s1': 0.509}, {'s2': 0.491}], 'a2': [{'s1': 0.31}, {'s3': 0.69}]}}, 's2': {'reward': 10, 'actions': {'a1': [{'s1': 0.4}, {'s2': 0.3}, {'s3': 0.3}], 'a2': [{'s2': 0.5}, {'s3': 0.5}]}}, 's3': {'reward': -5, 'actions': {'a1': [{'s1': 0.3}, {'s2': 0.3}, {'s3': 0.4}], 'a2': [{'s1': 0.2}, {'s2': 0.8}]}}}
[[5], [10], [-5]]
{'s1': 0, 's2': 1, 's3': 2}
{0: 's1', 1: 's2', 2: 's3'}
s1 ==> [5, 10.964, 14.492, 17.621, 20.004, 21.947, 23.489, 24.726, 25.714, 26.505, 27.138, 27.644, 28.049, 28.373, 28.632, 28.84, 29.006, 29.139, 29.245, 29.33]
s2 ==> [10, 12.8, 17.108, 19.93, 22.406, 24.317, 25.869, 27.103, 28.092, 28.883, 29.516, 30.022, 30.427, 30.751, 31.01, 31.217, 31.384, 31.516, 31.623, 31.708]
s3 ==> [-5, 2.2, 4.946, 8.268, 10.575, 12.54, 14.074, 15.